<a href="https://colab.research.google.com/github/devyulbae/AIClass/blob/main/04_Softmax_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.15.0'

# Softmax classification

* 임의의 Dataset 준비
* 3개의 클래스로 분류할 데이터 준비


In [6]:
x_data = [[1., 2., 1., 1.],
          [2., 1., 3., 2.],
          [3., 1., 3., 4.],
          [4., 1., 5., 5.],
          [1., 7., 5., 5.],
          [1., 2., 5., 6.],
          [1., 6., 6., 6.],
          [1., 7., 7., 7.]] # 8x4
y_data = [[0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 1., 0.],
          [0., 1., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]] # 8x3

x_test = [[1., 1., 1., 1.]]
y_test = [[0., 0., 1.]]


## 임의의 Data를 이용해서 3개의 클래스를 가지는 데이터셋 생성

In [7]:
#dataset을 선언합니다.
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
dataset = dataset.repeat(1).batch(8)

nb_classes = 3 # class의 개수입니다.

print(tf.Variable(x_data))
print(tf.Variable(y_data))

<tf.Variable 'Variable:0' shape=(8, 4) dtype=float32, numpy=
array([[1., 2., 1., 1.],
       [2., 1., 3., 2.],
       [3., 1., 3., 4.],
       [4., 1., 5., 5.],
       [1., 7., 5., 5.],
       [1., 2., 5., 6.],
       [1., 6., 6., 6.],
       [1., 7., 7., 7.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(8, 3) dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)>


In [8]:
#Weight and bias setting
W = tf.Variable(tf.random.normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')
variables = [W, b]

tf.print(W,b)

[[0.101562597 -1.23245883 -2.38226604]
 [0.444608241 0.649644434 -0.722008765]
 [1.50522554 -1.18143976 -1.22825968]
 [-0.44749102 -0.164771929 1.02197492]] [-0.427955598 1.42194891 -1.01549339]


# 가설 설정

* 가설에서 예측한 값들을 이용해 예측값들을 확률로 표현한다.

## $$ y_k = \frac{exp(x_k)}{\sum_{i=1}^{n}exp(x_i)}  $$

In [9]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis_softmax(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

tf.print(hypothesis_softmax(tf.Variable(x_data)))

[[0.813427567 0.185539216 0.0010331762]
 [0.999692619 0.000301130262 6.24554059e-06]
 [0.999850512 0.000139646945 9.84662256e-06]
 ...
 [0.999944627 2.01740895e-05 3.51710878e-05]
 [0.999996841 3.12030693e-06 2.15005169e-08]
 [0.999999583 3.46117304e-07 1.89164373e-09]]


## 가설을 검증할 Cross Entropy 함수를 정의합니다

$$
\begin{align}
cost(h(x),y) & = −\sum_{n=1}^{n} Y log(h(x))
\end{align}
$$

In [10]:
def loss_fn(hypothesis, labels):
    loss = tf.keras.losses.categorical_crossentropy(labels, hypothesis)
    # tf.keras.losses.binary_crossentropy(labels, hypothesis) # 이진분류용 CE Loss
    return loss

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.01)

### 학습 진행

In [11]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(hypothesis_softmax(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, tf.reduce_mean(loss_value.numpy()).numpy()))

Iter: 0, Loss: 8.3210
Iter: 100, Loss: 0.6846
Iter: 200, Loss: 0.4935
Iter: 300, Loss: 0.4190
Iter: 400, Loss: 0.3760
Iter: 500, Loss: 0.3458
Iter: 600, Loss: 0.3223
Iter: 700, Loss: 0.3029
Iter: 800, Loss: 0.2862
Iter: 900, Loss: 0.2717
Iter: 1000, Loss: 0.2587
Iter: 1100, Loss: 0.2470
Iter: 1200, Loss: 0.2364
Iter: 1300, Loss: 0.2266
Iter: 1400, Loss: 0.2177
Iter: 1500, Loss: 0.2095
Iter: 1600, Loss: 0.2018
Iter: 1700, Loss: 0.1947
Iter: 1800, Loss: 0.1881
Iter: 1900, Loss: 0.1819
Iter: 2000, Loss: 0.1761
Iter: 2100, Loss: 0.1706
Iter: 2200, Loss: 0.1655
Iter: 2300, Loss: 0.1607
Iter: 2400, Loss: 0.1561
Iter: 2500, Loss: 0.1518
Iter: 2600, Loss: 0.1477
Iter: 2700, Loss: 0.1438
Iter: 2800, Loss: 0.1401
Iter: 2900, Loss: 0.1366
Iter: 3000, Loss: 0.1332
Iter: 3100, Loss: 0.1300
Iter: 3200, Loss: 0.1270
Iter: 3300, Loss: 0.1241
Iter: 3400, Loss: 0.1213
Iter: 3500, Loss: 0.1186
Iter: 3600, Loss: 0.1161
Iter: 3700, Loss: 0.1136
Iter: 3800, Loss: 0.1113
Iter: 3900, Loss: 0.1090
Iter: 4000, 

## Sample 데이터를 넣고 테스트해봅시다.

In [12]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis_softmax(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

tf.Tensor([[0.00112876 0.04634704 0.9525242 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


## 데이터를 이용해서 예측

In [13]:
b = hypothesis_softmax(x_test)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_test, 1)) # matches with y_data


tf.Tensor([[2.8657442e-07 4.7645762e-04 9.9952328e-01]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)
tf.Tensor([2], shape=(1,), dtype=int64)


In [14]:
def accuracy_fn(hypothesis, labels):
    hypothesis = tf.argmax(hypothesis)
    hypothesis = tf.cast(hypothesis, dtype=tf.float32)
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    labels = tf.argmax(labels)
    labels = tf.cast(labels, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

test_acc = accuracy_fn(hypothesis_softmax(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 1.0000
